# Análisis de Series de Tiempo II

### Lectura y preparación de la serie de tiempo

---

Los datos corresponden a la temperaturas de superficie terrestre (LST) de diferentes tipo de cobertura (pino y estepa) de 3 pares de lotes localizados en la Patagonia cordillerana argentina, para 5 años consecutivos (2020-2024).
Los datos se obtuvieron a partir de información satelital con un script en GEE.


### 1. Lectura y preparación de los datos

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

`MODIS_Aqua_Terra.csv` contiene los datos crudos generados en Google Earth Engine.

In [2]:
df_aquaterra = pd.read_csv("csv/MODIS_Aqua_Terra_diario.csv",parse_dates=["fecha"])
df_aquaterra=df_aquaterra.dropna()
df_aquaterra.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5421 entries, 0 to 7251
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   system:index  5421 non-null   object        
 1   LST_Day       5421 non-null   float64       
 2   LST_Night     5421 non-null   float64       
 3   especie       5421 non-null   object        
 4   fecha         5421 non-null   datetime64[ns]
 5   satelite      5421 non-null   object        
 6   .geo          5421 non-null   object        
dtypes: datetime64[ns](1), float64(2), object(4)
memory usage: 338.8+ KB


In [3]:
df_aquaterra

,system:index,LST_Day,LST_Night,especie,fecha,satelite,.geo
0,1_1_1_2020_01_01,33.799010,13.653526,pino,2020-01-01,Aqua,"{""type"":""MultiPoint"",""coordinates"":[]}"
1,1_1_1_2020_01_02,34.932596,12.848341,pino,2020-01-02,Aqua,"{""type"":""MultiPoint"",""coordinates"":[]}"
2,1_1_1_2020_01_03,33.788120,13.174737,pino,2020-01-03,Aqua,"{""type"":""MultiPoint"",""coordinates"":[]}"
3,1_1_1_2020_01_04,33.781229,11.430565,pino,2020-01-04,Aqua,"{""type"":""MultiPoint"",""coordinates"":[]}"
9,1_1_1_2020_01_10,26.057463,4.503644,pino,2020-01-10,Aqua,"{""type"":""MultiPoint"",""coordinates"":[]}"
...,...,...,...,...,...,...,...
7246,2_2024_12_26,35.454205,17.012239,estepa,2024-12-26,Terra,"{""type"":""MultiPoint"",""coordinates"":[]}"
7247,2_2024_12_27,33.084581,14.427856,estepa,2024-12-27,Terra,"{""type"":""MultiPoint"",""coordinates"":[]}"
7249,2_2024_12_29,35.380471,14.531277,estepa,2024-12-29,Terra,"{""type"":""MultiPoint"",""coordinates"":[]}"
7250,2_2024_12_30,28.604990,11.765555,estepa,2024-12-30,Terra,"{""type"":""MultiPoint"",""coordinates"":[]}"


In [4]:
# agrupo los valores de Aqua y Terra para generar un df con 1 fila por fecha y tipo de cobertura
df = df_aquaterra.groupby(['fecha', 'especie'])[['LST_Day', 'LST_Night']].mean().reset_index()
df['LST'] = (df['LST_Day'] + df['LST_Night'])/2  # el promedio de las temperaturas
df['mes'] = df['fecha'].dt.month
df

,fecha,especie,LST_Day,LST_Night,LST,mes
0,2020-01-01,estepa,35.433453,12.923966,24.178709,1
1,2020-01-01,pino,33.666374,14.703904,24.185139,1
2,2020-01-02,estepa,36.195872,12.885076,24.540474,1
3,2020-01-02,pino,31.342840,13.202792,22.272816,1
4,2020-01-03,estepa,35.245773,12.973098,24.109435,1
...,...,...,...,...,...,...
3082,2024-12-29,pino,32.029511,13.523824,22.776668,12
3083,2024-12-30,estepa,27.980279,10.187533,19.083906,12
3084,2024-12-30,pino,25.945808,10.862246,18.404027,12
3085,2024-12-31,estepa,25.818976,8.286723,17.052850,12


In [5]:
# Pivoteo los datos
df_temp = (
    df_aquaterra
    .melt(id_vars=['fecha', 'especie'], value_vars=['LST_Day', 'LST_Night'],
          var_name='dia_noche', value_name='valor')
    .pivot_table(index=['fecha', 'dia_noche'], columns='especie', values='valor', aggfunc='mean')
    .reset_index()
)
df_temp['mes'] = df_temp['fecha'].dt.month
df_temp['delta'] = df_temp['estepa'] - df_temp['pino']
df_temp

especie,fecha,dia_noche,estepa,pino,mes,delta
0,2020-01-01,LST_Day,35.433453,33.666374,1,1.767079
1,2020-01-01,LST_Night,12.923966,14.703904,1,-1.779938
2,2020-01-02,LST_Day,36.195872,31.342840,1,4.853032
3,2020-01-02,LST_Night,12.885076,13.202792,1,-0.317715
4,2020-01-03,LST_Day,35.245773,31.284172,1,3.961601
...,...,...,...,...,...,...
3135,2024-12-29,LST_Night,13.086844,13.523824,12,-0.436980
3136,2024-12-30,LST_Day,27.980279,25.945808,12,2.034471
3137,2024-12-30,LST_Night,10.187533,10.862246,12,-0.674713
3138,2024-12-31,LST_Day,25.818976,20.606843,12,5.212133


In [6]:
# Creo un df con las temperaturas diurnas
df_dia = df_temp[df_temp['dia_noche'] == 'LST_Day']
df_dia=df_dia.drop(columns='dia_noche')
df_dia.head(10)

especie,fecha,estepa,pino,mes,delta
0,2020-01-01,35.433453,33.666374,1,1.767079
2,2020-01-02,36.195872,31.342840,1,4.853032
4,2020-01-03,35.245773,31.284172,1,3.961601
6,2020-01-04,38.821326,31.450303,1,7.371023
8,2020-01-05,NaN,17.944593,1,NaN
10,2020-01-07,25.590980,19.523039,1,6.067942
12,2020-01-10,27.450967,22.472114,1,4.978853
14,2020-01-11,36.205996,28.634072,1,7.571923
16,2020-01-12,35.113288,30.692833,1,4.420455
18,2020-01-13,37.252728,31.058509,1,6.194218


Completo con las fechas que faltan.

In [7]:
df_dia = df_dia.set_index('fecha')
df_dia = df_dia.asfreq('D') # genera filas nuevas para los días faltantes
df_dia = df_dia.reset_index()
df_dia['mes'] = df_dia['fecha'].dt.month
df_dia.head(10)

especie,fecha,estepa,pino,mes,delta
0,2020-01-01,35.433453,33.666374,1,1.767079
1,2020-01-02,36.195872,31.342840,1,4.853032
2,2020-01-03,35.245773,31.284172,1,3.961601
3,2020-01-04,38.821326,31.450303,1,7.371023
4,2020-01-05,NaN,17.944593,1,NaN
5,2020-01-06,NaN,NaN,1,NaN
6,2020-01-07,25.590980,19.523039,1,6.067942
7,2020-01-08,NaN,NaN,1,NaN
8,2020-01-09,NaN,NaN,1,NaN
9,2020-01-10,27.450967,22.472114,1,4.978853


### Exportación

In [8]:
# Guardo los datos para su procesamiento
df.to_csv('csv/ts_diario.csv', index=False)
df_dia.to_csv('csv/ts_d_diario.csv', index=False)